# Web Scraping Assignment-3

In [1]:
# import necessary libraries
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import pandas as pd
from selenium.common.exceptions import NoSuchElementException,TimeoutException
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs

import warnings
warnings.filterwarnings('ignore')

# 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [55]:
# open chrome and load amazon webpage
driver=webdriver.Chrome()
driver.get('http://www.amazon.in/')

# take input from user to search the keyword
WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')))
enter_kw=driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')
enter_kw.send_keys(input('Enter product to be searched: '))
search=driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input').click()

Enter product to be searched: headphones


# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. 
Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [57]:
# get product urls from first 3 pages
pdt_url=[]
for page in range(0,3):
    links=driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
    for i in links:
        pdt_url.append(i.get_attribute('href'))
        
    try:
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//span[@class="s-pagination-strip"]/a[3]')))
        next_button=driver.find_element(By.XPATH,'//span[@class="s-pagination-strip"]/a[3]')
        next_button.click()

    except NoSuchElementException:
        print('Timeout occurred or No next button found.')
        break

print('Number of urls extracted:',len(pdt_url), '\n\n' ,pdt_url)


Number of urls extracted: 28 

 ['https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo4ODY1MzQwMzg1NzE1MTA2OjE3MTMyNzY4OTU6c3BfYXRmOjMwMDA2Mzk3ODM5MTMzMjo6MDo6&url=%2FHAMMER-Bash-Max-Bluetooth-Headphones%2Fdp%2FB0CBFZYZM6%2Fref%3Dsr_1_1_sspa%3Fdib%3DeyJ2IjoiMSJ9.cr8fttXOsYlU2Yp2oPY96jIk_Aib2yrdRsKZZG4DCMnxV4G8eGnusQBLYHxaTLhaL-4QC6D6jBI9OorM6JH1AVKsLPKlpdQYu8lr6VDWg7vvbhgVhA-3d3P3RGXPYvLx0iSGvcXDub4sJiOUJSTYwIpY-paCoxE9bnvw1nIHebokBAf1Cw2ce6wz6hTEnhw_ANpp5zHIBeVOux-FWL7sXCDB1jDmnh6z7yPr2i9yvN4.U7XyhFiCVLw1YZPlrQNPhnZBbOFWB_tCTa9ZzvIndK4%26dib_tag%3Dse%26keywords%3Dheadphones%26qid%3D1713276895%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1', 'https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo4ODY1MzQwMzg1NzE1MTA2OjE3MTMyNzY4OTU6c3BfYXRmOjMwMDA2NDYzODg5MTUzMjo6MDo6&url=%2FBoult-Playtime-Bluetooth-Wireless-Lavender%2Fdp%2FB0CHVWYZV8%2Fref%3Dsr_1_2_sspa%3Fdib%3DeyJ2IjoiMSJ9.cr8fttXOsYlU2Yp2oPY96jIk_Aib2yrdRsKZZG4DCMnxV4G8eGnusQBLYHxaTLhaL-4QC6D6jBI9OorM6JH1AVKsLPKlpdQYu8lr6VDWg7v

In [58]:
# ensure there are no replications
from collections import Counter
counts=Counter(pdt_url)
dup=[items for items, count in counts.items() if count>1]
len(dup)

0

In [65]:
# scrape product details from first 3 pages
brand=[]
pdt=[]
price=[]
ret=[]
delivery=[]
avail=[]

for i in pdt_url:
    driver.get(i)
    time.sleep(4)
    
    # scrape brand names
    try:
        names=driver.find_elements(By.XPATH,'//table[@class="a-normal a-spacing-micro"]/tbody/tr[1]/td[2]')
        if names:
            for name in names:
                brand.append(name.text.strip())
        else:
            brand.append('-')
    except NoSuchElementException:
        brand.append('-')
    
    # scrape product names
    try:
        pdts=driver.find_elements(By.XPATH,'//div[@class="a-section a-spacing-none"]/h1/span')
        if pdts:
            for item in pdts:
                pdt.append(item.text.strip())
        else:
            pdt.append('-')
    except NoSuchElementException:
        pdt.append('-')
    
    # scrape product price
    try:
        costs=driver.find_elements(By.XPATH,'//span[@class="a-price aok-align-center"]')
        if costs:
            for cost in costs:
                price.append(cost.text.replace('\n','.').replace('₹',''))
        else:
            price.append('-')
    except NoSuchElementException:
        price.append('-')
    
    # scrape product delivery
    try:
        deli=driver.find_elements(By.XPATH,'//div[@class="a-spacing-base"][1]')
        if deli:
            for d in deli:
                delivery.append(d.text.split('.')[0])
        else:
            delivery.append('-')
    except NoSuchElementException:
        delivery.append('-')
    
    # scrape product Return/Exchange
    try:
        returns=driver.find_element(By.XPATH,'//span[@class="a-size-small a-color-link a-text-normal"][1]')
        ret.append(returns.text.strip())
            
    except NoSuchElementException:
        ret.append('-')
    except AttributeError:
        ret.append('-')
    
    # scrape product availability
    try:
        availability=driver.find_elements(By.XPATH,'//span[@class="a-size-medium a-color-success"]')
        if availability:
            for a in availability:
                avail.append(a.text.strip())
        else:
            avail.append('-')
    except NoSuchElementException:
        avail.append('-')


print('Number of brand names extracted:',len(brand), '\n\n' ,brand[:5], '\n\n','*'*75)
print('Number of product names extracted:',len(pdt), '\n\n' ,pdt[:5], '\n\n','*'*75)
print('Number of product price extracted:',len(price), '\n\n' ,price[:5], '\n\n','*'*75)
print('Number of product delivery data extracted:',len(delivery), '\n\n' ,delivery[:5], '\n\n','*'*75)
print('Number of product Return/Exchange extracted:',len(ret), '\n\n' ,ret[:5], '\n\n','*'*75)
print('Number of product availability extracted:',len(avail), '\n\n' ,avail[:5], '\n\n','*'*75)

Number of brand names extracted: 28 

 ['HAMMER', 'Boult', 'boAt', 'Logitech', 'Sony'] 

 ***************************************************************************
Number of product names extracted: 28 

 ['HAMMER Bash Max Over The Ear Wireless Bluetooth Headphones with Mic, Touch Control, Deep Bass, Upto 18 Hours Playtime, Bluetooth 5.3, Workout/Travel (Black)', 'Boult Audio Z40 Pro with 100H Playtime, Quad Mic ENC, 45ms Low Latency Gaming, Premium Rubber Grip Case, 13mm Bass Drivers, Made in India TWS Bluetooth 5.3 Truly Wireless in Ear Earbuds (Lavender)', 'boAt Rockerz 450R On-Ear Headphones with 15 Hours Battery, 40mm Drivers, Padded Ear Cushions, Easy Access Controls and Voice Assistant(Aqua Blue)', 'Logitech H111 Wired On Ear Headphones With Mic Black', 'Sony WH-CH520, Wireless On-Ear Bluetooth Headphones with Mic, Upto 50 Hours Playtime, DSEE Upscale, Multipoint Connectivity/Dual Pairing,Voice Assistant App Support for Mobile Phones (Black)'] 

 ******************************

In [66]:
print(len(brand),len(pdt),len(price),len(delivery),len(ret),len(avail),len(pdt_url))

28 28 28 28 28 28 28


In [67]:
# saving to dataframe
amazon_df=pd.DataFrame({'Product brand': brand,'Product name': pdt,'Price (₹)': price,'Expected delivery':delivery,'Return/Exchange':ret,'Availability':avail,'Product link': pdt_url})
amazon_df

,Product brand,Product name,Price (₹),Expected delivery,Return/Exchange,Availability,Product link
0,HAMMER,HAMMER Bash Max Over The Ear Wireless Bluetoot...,"2,399.00","FREE delivery Saturday, 20 April",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Boult,"Boult Audio Z40 Pro with 100H Playtime, Quad M...","1,499.00","FREE delivery Friday, 19 April",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,boAt,boAt Rockerz 450R On-Ear Headphones with 15 Ho...,"1,499.00","FREE delivery Friday, 19 April",7 days Replacement,In stock,https://www.amazon.in/boAt-Rockerz-450-Headpho...
3,Logitech,Logitech H111 Wired On Ear Headphones With Mic...,699.00,"FREE delivery Friday, 19 April",7 days Replacement,In stock,https://www.amazon.in/Logitech-H111-Stero-Head...
4,Sony,"Sony WH-CH520, Wireless On-Ear Bluetooth Headp...","4,490.00","FREE delivery Monday, 22 April",7 days Service Centre Replacement,In stock,https://www.amazon.in/Sony-Bluetooth-Headphone...
5,soundcore,"Soundcore H30i Wireless On-Ear Headphones, Fol...","2,999.00","FREE delivery Friday, 19 April",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
6,soundcore,soundcore by Anker H30i Wireless On-Ear Headph...,"2,799.00","FREE delivery Friday, 19 April",7 days Returnable,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
7,boAt,boAt Rockerz 450 Bluetooth On Ear Headphones w...,"1,499.00","FREE delivery Friday, 19 April",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
8,Philips Audio,"Philips Wireless On Ear Headphones TAH8506BK, ...","8,999.00","FREE delivery Friday, 19 April",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
9,Boult,Boult Audio Z40 True Wireless in Ear Earbuds w...,"1,399.00","FREE delivery Friday, 19 April",7 days Replacement,In stock,https://aax-eu.amazon.in/x/c/RKvdainxR5svcPJcD...


In [68]:
# saving to csv
amazon_df.to_csv('amazon_search_results_1.csv',index=False)

In [69]:
driver.quit()

# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [7]:
# open chrome and load google
driver=webdriver.Chrome()
driver.get('http://www.google.in/')

In [16]:
# click on google image search
image_searcher=driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div/div/div/div/div[1]/div/div[2]/a').click()

In [17]:
search_bar=driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea')
search_bar.clear()
search_bar.send_keys('fruits')
search=driver.find_element(By.XPATH, '//span[@class="z1asCe MZy1Rb"]').click()


In [25]:
current_url=driver.current_url

In [27]:
from bs4 import BeautifulSoup as bs
import requests
response=requests.get(current_url)

isoup=bs(response.content,'html.parser')

pic=[]
for i in isoup.find_all('img'):
    pic.append(i['src'])
pic

['/images/branding/searchlogo/1x/googlelogo_desk_heirloom_color_150x55dp.gif',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQnqaJidY3yxsv_D47D1Erbglu4o0qTm4dnfwykp-77BwGrOz-M9DKSpJIF3g&s',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ86pCpQFnZQzJlSUCJ-i04qECrYBku6HdYZj-wObv4baRAki2NSYnimmzNpw&s',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ1c1DAfgF3XONwSuKO3OrytUnolbKyE9ZoLSuQQCGb8udf05FY2C3bUobXqg&s',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTY-mlSWC0cAH0cqukdmKr1tTgF8aW1IId4hE-mZ6OeiggEVyFNsjqaU0_Dvg&s',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQznB87BHxCHe9otYj5XbfzPXitr8gEFlXS6O4HikGC7DEb--21bW4nxtUs5UA&s',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSDLE8vytJGzvuGZtgQCGWhqUdCcYhjQH4DzQph9ui9YqbiXO5jn_PNVzWFUUU&s',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS70zWXpTuW4Z98mJT-zWlx-QeXJCOFeeh1DqJcePMEIpFyuZdis07PqGSAEw&s',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTYtsivAk5AkooQTV

As the image urls were in encrypted format, I couldn't complete this question. Instead I have used pygoogle_image to download images as an alternative.

In [43]:
!pip install pygoogle_image

In [42]:
# Since I couldn't scrape using selenium, I have downloaded using pygoogle_image

from pygoogle_image import image as pi
keywords=['fruits', 'cars','Machine Learning', 'Guitar', 'Cakes']
for key in keywords:
    pi.download(key,limit=10)

[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%


In [179]:
driver.quit()

# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. 
Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. 

Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [103]:
# open chrome and load flipkart
driver=webdriver.Chrome()
driver.get('http://www.flipkart.in/')

In [104]:
# search for keyword
keyword=input('Enter name of smartphone to search: ')
enter=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div/div/div/div/div[1]/div/div[1]/div/div[1]/div[1]/header/div[1]/div[2]/form/div/div/input')
enter.send_keys(keyword)
search=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div/div/div/div/div[1]/div/div[1]/div/div[1]/div[1]/header/div[1]/div[2]/form/div/button')
search.click()

Enter name of smartphone to search: oneplus nord


In [105]:
# scrape Product URL
pdturl=[]
try:
    for i in driver.find_elements(By.XPATH,'//a[@class="CGtC98" ]'):
        pdturl.append(i.get_attribute('href'))
except NoSuchElementException:
    pdturl.append('-')
    
print('Number of Smartphone product url extracted:', len(pdturl), '\n\n', pdturl, '\n\n')  

Number of Smartphone product url extracted: 24 

 ['https://www.flipkart.com/oneplus-nord-ce-3-lite-5g-chromatic-gray-256-gb/p/itm2cd5a4e659035?pid=MOBGZJ45PS4AQ7BA&lid=LSTMOBGZJ45PS4AQ7BASHPAA2&marketplace=FLIPKART&q=oneplus+nord&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=6262eed9-3061-4025-baa7-caa3b7e3a26f.MOBGZJ45PS4AQ7BA.SEARCH&ppt=hp&ppn=homepage&ssid=po3rtw8r8g0000001713614765565&qH=e8b736bf4fa84421', 'https://www.flipkart.com/oneplus-nord-ce-3-lite-5g-pastel-lime-256-gb/p/itm2cd5a4e659035?pid=MOBGZJ3ZFQ8AKKRS&lid=LSTMOBGZJ3ZFQ8AKKRSXHP4UY&marketplace=FLIPKART&q=oneplus+nord&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=6262eed9-3061-4025-baa7-caa3b7e3a26f.MOBGZJ3ZFQ8AKKRS.SEARCH&ppt=hp&ppn=homepage&ssid=po3rtw8r8g0000001713614765565&qH=e8b736bf4fa84421', 'https://www.flipkart.com/oneplus-nord-ce-3-lite-5g-chromatic-gray-128-gb/p/itm2cd5a4e659035?pid=MOBGZJ42KHUZZKMN&lid=LSTMOBGZJ42KHUZZKMNNZJMSB&marketplace=FLIPKART&q

In [47]:
pdturl[23]

'https://www.flipkart.com/lava-blaze-glass-green-64-gb/p/itmbe44c3b1ad1a3?pid=MOBGEUHVFUZZBHJW&lid=LSTMOBGEUHVFUZZBHJWC4IHKR&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_24&otracker=search&otracker1=search&fm=organic&iid=c589d2e5-7dea-48e5-8273-2be232adc27e.MOBGEUHVFUZZBHJW.SEARCH&ppt=hp&ppn=homepage&ssid=pk2uvj0u680000001713606509220&qH=9b26a23b2cff510d'

In [106]:
# scrape Smartphone brand

brand=[]

for i in pdturl:
    try:
        driver.get(i)
    except TimeoutException:
        print(f'Timeout occurred while loading {i}')
        driver.refresh()  

    try:
        brands= driver.find_element(By.XPATH,'//span[@class="VU-ZEz"]')
        b=' '.join(brands.text.split()[0:2])
        brand.append(b)
    except NoSuchElementException:
        brand.append('-')
        
print('Number of Smartphone brands extracted:', len(brand), '\n\n', brand, '\n\n')

Number of Smartphone brands extracted: 24 

 ['OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord', 'OnePlus Nord'] 




In [107]:
# scrape Smartphone name

name=[]

for i in pdturl:
    try:
        driver.get(i)
    except TimeoutException:
        print(f'Timeout occurred while loading {i}')
        driver.refresh()  

    try:
        names = driver.find_element(By.XPATH, '//table[@class="_0ZhAN9"]/tbody/tr[td[contains(text(), "Model Name")]]/td[2]')
        name.append(names.text)
    except NoSuchElementException:
        name.append('-')
        
print('Number of Smartphone names extracted:', len(name), '\n\n', name, '\n\n')

Number of Smartphone names extracted: 24 

 ['Nord CE 3 Lite 5G', 'Nord CE 3 Lite 5G', 'Nord CE 3 Lite 5G', 'Nord CE 3 Lite 5G', 'Nord CE4', 'Nord CE4', 'Nord CE4', 'Nord CE4', 'Nord 2T 5G', 'Nord 2T 5G', 'Nord CE 2 5G', 'Nord 2T 5G', 'Nord CE 2 5G', 'Nord 2 5G', 'Nord 2 5G', 'Nord 2 5G', 'Nord 2T 5G', 'Nord 2T 5G', 'Nord 3 5G', 'Nord 3 5G', 'Nord CE3 5G', 'Nord CE3 5G', 'Nord 2T 5G', 'Nord CE 2 5G'] 




In [125]:
# scrape Smartphone color

color=[]

for i in pdturl:
    try:
        driver.get(i)
    except TimeoutException:
        print(f'Timeout occurred while loading {i}')
        driver.refresh()  
        
    try:
        colors= driver.find_element(By.XPATH,'//table[@class="_0ZhAN9"]/tbody/tr[td[contains(text(), "Color")]]/td[2]')
        color.append(colors.text)
    except NoSuchElementException:
        color.append('-')
        
print('Number of Smartphone color extracted:', len(color), '\n\n', color, '\n\n')

Number of Smartphone color extracted: 24 

 ['Chromatic Gray', 'Pastel Lime', 'Chromatic Gray', 'Pastel Lime', 'Dark Chrome', 'Celadon Marble', 'Celadon Marble', 'Dark Chrome', 'Jade Fog', 'Gray Shadow', 'Bahama Blue', 'Jade Fog', 'Bahama Blue', 'Gray Sierra', 'Gray Sierra', 'Blue Haze', 'Jade Fog', 'Gray Shadow', 'Misty Green', 'Tempest Gray', 'Grey Shimmer', 'Aqua Surge', 'Gray Shadow', 'Gray Mirror'] 




In [108]:
# scrape Smartphone ram & rom

ram=[]
rom=[]

for i in pdturl:
    try:
        driver.get(i)
    except TimeoutException:
        print(f'Timeout occurred while loading {i}')
        driver.refresh()
    
    try:
        r= driver.find_element(By.XPATH,'//div[@class="xFVion"]/ul/li[1]')
        texts=r.text.split('|')
        ram.append(texts[0])
        rom.append(texts[1])
    except NoSuchElementException:
        ram.append('-')
        rom.append('-')

print('Number of ROM data extracted:', len(rom), '\n\n', rom, '\n\n')
print('Number of RAM data extracted:', len(ram), '\n\n', ram, '\n\n')

Number of ROM data extracted: 24 

 [' 256 GB ROM', ' 256 GB ROM', ' 128 GB ROM', ' 128 GB ROM', ' 256 GB ROM', ' 256 GB ROM', ' 128 GB ROM', ' 128 GB ROM', ' 128 GB ROM', ' 128 GB ROM', ' 128 GB ROM', ' 128 GB ROM', ' 128 GB ROM', ' 256 GB ROM', ' 128 GB ROM', ' 128 GB ROM', ' 256 GB ROM', ' 256 GB ROM', ' 256 GB ROM', ' 256 GB ROM', ' 256 GB ROM', ' 256 GB ROM', ' 128 GB ROM', ' 128 GB ROM'] 


Number of RAM data extracted: 24 

 ['8 GB RAM ', '8 GB RAM ', '8 GB RAM ', '8 GB RAM ', '8 GB RAM ', '8 GB RAM ', '8 GB RAM ', '8 GB RAM ', '8 GB RAM ', '8 GB RAM ', '8 GB RAM ', '8 GB RAM ', '8 GB RAM ', '12 GB RAM ', '8 GB RAM ', '8 GB RAM ', '12 GB RAM ', '12 GB RAM ', '16 GB RAM ', '16 GB RAM ', '12 GB RAM ', '12 GB RAM ', '8 GB RAM ', '8 GB RAM '] 




In [109]:
# scrape Smartphone Primary Camera

pcam=[]

driver.set_page_load_timeout(10)
for i in pdturl:
    try:
        driver.get(i)
        time.sleep(2)
    except TimeoutException:
        print(f'Timeout occurred while loading {i}')
        driver.refresh()
        
    try:
        read_more=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/button')
        read_more.click()
        #time.sleep(2)
        #WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//table[@class="_0ZhAN9"]/tbody/tr[td[contains(text(), "Primary Camera")]]/td[2]')))        
        pcams= driver.find_element(By.XPATH,'//table[@class="_0ZhAN9"]/tbody/tr[td[contains(text(), "Primary Camera")]]/td[2]')
        pcam.append(pcams.text)
        
    except NoSuchElementException:
        pcam.append('-')
        
print('Number of Smartphone Primary Camera data extracted:', len(pcam), '\n\n', pcam, '\n\n')

Number of Smartphone Primary Camera data extracted: 24 

 ['108MP Rear Camera', '108MP Rear Camera', '108MP Rear Camera', '108MP Rear Camera', 'Yes', 'Yes', 'Yes', 'Yes', '-', '-', '-', '50MP Rear Camera', '-', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '-', '50MP Rear Camera', '-', '-', '-', '-', '-', '-'] 




In [ ]:
read_more=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/button')
read_more.click()
time.sleep(2)

In [110]:
# scrape Smartphone Secondary Camera

scam=[]
driver.set_page_load_timeout(10)
for i in pdturl:
    try:
        driver.get(i)
    except TimeoutException:
        print(f'Timeout occurred while loading {i}')
        driver.refresh()
    try:
        read_more=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/button')
        read_more.click()
        time.sleep(2)
        scams= driver.find_element(By.XPATH,'//table[@class="_0ZhAN9"]/tbody/tr[td[contains(text(), "Secondary Camera")]]/td[2]')
        scam.append(scams.text)
    except NoSuchElementException:
        scam.append('-')
        
print('Number of Smartphone Secondary Camera data extracted:', len(scam), '\n\n', scam, '\n\n')

Number of Smartphone Secondary Camera data extracted: 24 

 ['-', '-', '-', '-', 'Yes', 'Yes', 'Yes', 'Yes', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] 




In [111]:
# scrape Smartphone Display Size

dis=[]

for i in pdturl:
    try:
        driver.get(i)
    except TimeoutException:
        print(f'Timeout occurred while loading {i}')
        driver.refresh()
      

    
    try:
        read_more=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/button')
        read_more.click()
        time.sleep(2)          
        #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[td[contains(text(), "Display Size"]]/td[2]')))
        display = driver.find_element(By.XPATH, '//table[@class="_0ZhAN9"]/tbody/tr[td[contains(text(), "Display Size")]]/td[2]')
        dis.append(display.text)
    except NoSuchElementException:
        dis.append('-')
    except TimeoutException:
        dis.append('-')
        
print('Number of Smartphone Display Size data extracted:', len(dis), '\n\n', dis, '\n\n')

Number of Smartphone Display Size data extracted: 24 

 ['17.07 cm (6.72 inch)', '17.07 cm (6.72 inch)', '17.07 cm (6.72 inch)', '17.07 cm (6.72 inch)', '17.02 cm (6.7 inch)', '17.02 cm (6.7 inch)', '17.02 cm (6.7 inch)', '17.02 cm (6.7 inch)', '-', '-', '-', '17.02 cm (6.7 inch)', '-', '16.33 cm (6.43 inch)', '16.33 cm (6.43 inch)', '16.33 cm (6.43 inch)', '-', '17.02 cm (6.7 inch)', '-', '-', '-', '-', '-', '-'] 




In [127]:
# scrape Smartphone Battery Capacity

batt=[]

for i in pdturl:
    try:
        driver.get(i)
    except TimeoutException:
        print(f'Timeout occurred while loading {i}')
        driver.refresh()
      
       

    try:
        read_more=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/button')
        read_more.click()
        time.sleep(2) 
        battery= driver.find_element(By.XPATH,'//table[@class="_0ZhAN9"]/tbody/tr[td[contains(text(), "Battery Capacity")]]/td[2]')
        if battery:
            batt.append(battery.text)
        else:
            batt.append('-')
    except NoSuchElementException:
        batt.append('-')
        
print('Number of Smartphone Battery Capacity data extracted:', len(batt), '\n\n', batt, '\n\n')

Number of Smartphone Battery Capacity data extracted: 24 

 ['5000 mAh', '5000 mAh', '5000 mAh', '5000 mAh', '5500 mAh', '5500 mAh', '5500 mAh', '5500 mAh', '-', '-', '-', '4500 mAh', '-', '4500 mAh', '4500 mAh', '4500 mAh', '-', '4500 mAh', '-', '-', '-', '-', '-', '-'] 




In [122]:
# scrape Smartphone Price

price=[]
for i in pdturl:
    try:
        driver.get(i)
        time.sleep(3)
    except TimeoutException:
        driver.refresh()

    try:
        prices= driver.find_element(By.XPATH,'//div[@class="Nx9bqj CxhGGd"]')
        price.append(prices.text.replace('₹',''))
    except NoSuchElementException:
        price.append('-')

print('Number of Smartphone Battery Capacity data extracted:', len(price), '\n\n', price, '\n\n')

Number of Smartphone Battery Capacity data extracted: 24 

 ['19,427', '19,475', '17,414', '17,420', '26,259', '26,999', '24,546', '24,450', '27,499', '27,499', '23,945', '28,899', '22,899', '31,999', '27,999', '29,999', '28,999', '32,899', '29,999', '35,700', '27,037', '26,999', '27,000', '17,999'] 




In [128]:
print(len(brand),len(name),len(price),len(color),len(ram),len(rom),len(pcam),len(scam),len(dis),len(batt),len(pdturl))

24 24 24 24 24 24 24 24 24 24 24


In [129]:
# saving to dataframe
flipkart_df=pd.DataFrame({'Product brand': brand,'Product name': name,'Price (₹)': price,'Colour':color,'RAM storage':ram,'ROM storage':rom,'Primary Camera':pcam,'Secondary  Camera':scam,'Display Size':dis,'Battery Capacity':batt,'Product link': pdturl})
flipkart_df

,Product brand,Product name,Price (₹),Colour,RAM storage,ROM storage,Primary Camera,Secondary Camera,Display Size,Battery Capacity,Product link
0,OnePlus Nord,Nord CE 3 Lite 5G,"19,427",Chromatic Gray,8 GB RAM,256 GB ROM,108MP Rear Camera,-,17.07 cm (6.72 inch),5000 mAh,https://www.flipkart.com/oneplus-nord-ce-3-lit...
1,OnePlus Nord,Nord CE 3 Lite 5G,"19,475",Pastel Lime,8 GB RAM,256 GB ROM,108MP Rear Camera,-,17.07 cm (6.72 inch),5000 mAh,https://www.flipkart.com/oneplus-nord-ce-3-lit...
2,OnePlus Nord,Nord CE 3 Lite 5G,"17,414",Chromatic Gray,8 GB RAM,128 GB ROM,108MP Rear Camera,-,17.07 cm (6.72 inch),5000 mAh,https://www.flipkart.com/oneplus-nord-ce-3-lit...
3,OnePlus Nord,Nord CE 3 Lite 5G,"17,420",Pastel Lime,8 GB RAM,128 GB ROM,108MP Rear Camera,-,17.07 cm (6.72 inch),5000 mAh,https://www.flipkart.com/oneplus-nord-ce-3-lit...
4,OnePlus Nord,Nord CE4,"26,259",Dark Chrome,8 GB RAM,256 GB ROM,Yes,Yes,17.02 cm (6.7 inch),5500 mAh,https://www.flipkart.com/oneplus-nord-ce4-dark...
5,OnePlus Nord,Nord CE4,"26,999",Celadon Marble,8 GB RAM,256 GB ROM,Yes,Yes,17.02 cm (6.7 inch),5500 mAh,https://www.flipkart.com/oneplus-nord-ce4-cela...
6,OnePlus Nord,Nord CE4,"24,546",Celadon Marble,8 GB RAM,128 GB ROM,Yes,Yes,17.02 cm (6.7 inch),5500 mAh,https://www.flipkart.com/oneplus-nord-ce4-cela...
7,OnePlus Nord,Nord CE4,"24,450",Dark Chrome,8 GB RAM,128 GB ROM,Yes,Yes,17.02 cm (6.7 inch),5500 mAh,https://www.flipkart.com/oneplus-nord-ce4-dark...
8,OnePlus Nord,Nord 2T 5G,"27,499",Jade Fog,8 GB RAM,128 GB ROM,-,-,-,-,https://www.flipkart.com/oneplus-nord-2t-5g-ja...
9,OnePlus Nord,Nord 2T 5G,"27,499",Gray Shadow,8 GB RAM,128 GB ROM,-,-,-,-,https://www.flipkart.com/oneplus-nord-2t-5g-gr...


In [130]:
# saving to csv
flipkart_df.to_csv('flipkart_search_results_1.csv',index=False)

In [131]:
driver.quit()

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [129]:
def geo_coordinates(city):
    
    # open chrome and load google maps
    driver=webdriver.Chrome()
    driver.get('http://www.google.com/maps')
    time.sleep(3)
    
    # search by keyword
    enter=driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[2]/form/input')
    enter.send_keys(city)
    search=driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[2]/div[1]/button')
    search.click()
    
    # extract latitude and longitude using regex from current url after the search 
    url=driver.current_url
    coordinates=re.findall(r'@\d{1,2}\.\d+,\d{1,2}\.\d+',url)
    lat=coordinates[0].replace('@','').split(',')[0]
    long=coordinates[0].replace('@','').split(',')[1]
    driver.quit()
    
    print(f'Here are the coordinates of {city} city! \n\n Latitude of {city} is : {lat} \n Longitude of {city} is : {long}')

In [130]:
geo_coordinates('bangalore')

Here are the coordinates of bangalore city! 

 Latitude of bangalore is : 8.8178688 
 Longitude of bangalore is : 76.9753088


# 6. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [40]:
# open chrome and load digit
driver=webdriver.Chrome()
driver.get('http://www.digit.in/')
time.sleep(5)

In [42]:
# go to the best gaming laptop option
step1=driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[2]/div/nav/ul/li[2]').click()
step2=driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[2]/div/nav/ul/li[2]/ul/li[3]/ul/li[1]/a').click()
time.sleep(3)

In [46]:
# get urls of all best gaming laptop pages given in digit webpage
lap_url=[]
for i in driver.find_elements(By.XPATH,'//li[@id="menu-item-123493"][3]/ul/li/a[contains(text(),"AMD")]'):
    lap_url.append(i.get_attribute('href'))

lap_url

['https://www.digit.in/top-products/best-amd-gaming-laptops-with-nvidia-geforce-rtx-3050-4073.html',
 'https://www.digit.in/top-products/best-amd-gaming-laptop-with-16gb-ram-4072.html',
 'https://www.digit.in/top-products/best-amd-gaming-laptops-for-students-4071.html',
 'https://www.digit.in/top-products/best-amd-gaming-laptop-with-nvidia-geforce-rtx-3060-4070.html']

In [81]:
title=[]
details=[]
desc=[]

for i in lap_url:
    driver.get(i)
    
# scrape laptop name
    try:
        for a in driver.find_elements(By.XPATH,'//h3[@class="font130 mt0 mb10 mobilesblockdisplay "]/a'):
            title.append(a.text)
            
    except NoSuchElementException:
        title.append('-')

# scrape laptop details and split into separate lists
    try:
        for i in driver.find_elements(By.XPATH,'//div[@class="rh-flex-center-align"]/span[2]'):
            details.append(i.text)
            os=details[::4]
            disp=details[1::4]
            res=details[2::4]
            processor=details[3::4]
            
    except NoSuchElementException:
        title.append('-')
    
# scrape laptop description
    try:
        for i in driver.find_elements(By.XPATH,'//div[@class="post-inner-wrapper leftbarcalc"]/p'):
            desc.append(i.text.strip())
            
    except NoSuchElementException:
        desc.append('-')
            
    desc = list(filter(lambda x: x.strip(), desc))

In [86]:
# add the scraped data to dataframe
laptop_details=pd.DataFrame({'Laptop name':title,'Operating system':os,'Display size': disp,'Processor': processor,'Resolution':res,'Detailed product description':desc})
from IPython.display import display
display(laptop_details)

,Laptop name,Operating system,Display size,Processor,Resolution,Detailed product description
0,Lenovo IdeaPad Gaming 3 82K200X3IN Ryzen 7-580...,Windows 11 Home,15.6,5th Gen AMD Ryzen 7-5800,1920 x 1080,"On the budget side of things, the Lenovo IdeaP..."
1,Dell New G15-5515 D560805WIN9W | Ryzen 5-5600H...,Windows 10,15.6,AMD Ryzen 5-5600H,1920 x 1080,For people looking for a gaming laptop with hi...
2,HP Pavilion Gaming 15-Ec2145AX | Ryzen 5 5600H...,Windows 11 Home,15.6,AMD Ryzen 5 5600H,1920 x 1080,The HP Pavilion Gaming 15 is a good choice for...
3,Lenovo IdeaPad Gaming 3 15ACH6 Ryzen 5-5600H (...,Windows 11 Home,15.6,AMD Ryzen 5-5600H,1920 x 1080,This is another variant of the Lenovo IdeaPad ...
4,HP OMEN 15-en1037AX 5th Gen Ryzen 7-5800H (2021),Windows 10 Home,15.6,5th Gen AMD Ryzen 7-5800H,2560 x 1440,The main highlight of this HP Omen 15 is its p...
5,Acer Nitro 5 AN515-45-R3TC Ryzen 5-5600H (2021),Windows 10 Home,15.6,AMD Ryzen 5 Hexa Core 5600H,1920 x 1080,It is probably the most budget-friendly option...
6,Dell New G15-5515 D560730WIN9W Ryzen 7-5800H,Windows 11 Home,15.6,AMD Ryzen R7-5800H,1920 x 1080,All Dell laptops come with superior build qual...
7,Lenovo Ideapad Gaming 3 82K200X2IN | Ryzen 7-5...,Windows 11 Home,15.6,AMD Ryzen 7-5800H,1920 x 1080,"For the ultimate budget-focused buyers, the Le..."
8,HP Victus 16-E0362ax | Ryzen 7-5800H | RTX 306...,Windows 11 Home,16.1,5th Gen AMD Ryzen 7-5800H,2560 x 1440,This HP Victus 16.1 probably offers one of the...
9,Acer Nitro 5 AN515-45-R3TC Ryzen 5-5600H (2021),Windows 10 Home,15.6,AMD Ryzen 5 Hexa Core 5600H,1920 x 1080,This is probably the most value for money gami...


In [87]:
driver.quit()

# 7. Write a python program to scrape the details for all billionaires from www.forbes.com. 
Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [2]:
# open chrome and load digit

driver=webdriver.Chrome()
driver.get('http://www.forbes.com/')
time.sleep(3)

In [5]:
# select options to go to webpage of billionaires

step1=driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div[1]/div/div').click()

step2=driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div[1]/div/div[2]/ul/li[2]/div[1]')
from selenium.webdriver.common.action_chains import ActionChains
action=ActionChains(driver)
action.move_to_element(step2).perform()

step3=driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div[1]/div/div[2]/ul/li[2]/div[2]/div[3]/ul/li[1]').click()

In [94]:
# scrape table
table=[]
for i in driver.find_elements(By.XPATH,'//div[@role="row"]'):
    table.append(i.text.strip().splitlines())


rank=[]
name=[]
nw=[]
age =[]
citizen=[]
source=[]
ind=[]

for row in table:
    if len(row)==7:
        rank.append(row[0])
        name.append(row[1])
        nw.append(row[2])
        age.append(row[3])
        citizen.append(row[4])
        source.append(row[5])
        ind.append(row[6])
    else:
        print(f'{row} has different numer of elements than others')

print(rank,'\n\n',nw,'\n\n',name,'\n\n',age,'\n\n',citizen,'\n\n',source,'\n\n',ind)

['RANK', '1.', '2.', '3.', '4.', '5.', '6.', '7.', '8.', '9.', '10.', '11.', '12.', '13.', '14.', '15.', '16.', '17.', '18.', '19.', '20.', '21.', '22.', '23.', '24.', '25.', '26.', '27.', '27.', '29.', '30.', '31.', '32.', '33.', '34.', '35.', '35.', '37.', '38.', '39.', '40.', '40.', '42.', '43.', '44.', '45.', '46.', '46.', '48.', '48.', '50.', '51.', '52.', '53.', '54.', '55.', '56.', '57.', '58.', '59.', '60.', '60.', '62.', '63.', '64.', '65.', '66.', '66.', '68.', '69.', '70.', '71.', '71.', '73.', '73.', '75.', '76.', '76.', '76.', '79.', '80.', '81.', '81.', '83.', '84.', '85.', '86.', '87.', '88.', '89.', '90.', '91.', '92.', '92.', '94.', '94.', '94.', '97.', '98.', '99.', '100.', '101.', '102.', '103.', '104.', '104.', '104.', '107.', '108.', '109.', '110.', '111.', '111.', '113.', '113.', '115.', '115.', '117.', '118.', '118.', '120.', '121.', '122.', '122.', '124.', '125.', '126.', '126.', '126.', '129.', '130.', '131.', '132.', '132.', '132.', '135.', '135.', '137.', '13

In [95]:
# ensure length of all lists are same
print(len(rank),len(nw),len(name),len(age),len(citizen),len(source),len(ind))

201 201 201 201 201 201 201


In [96]:
billionaires_forbes=pd.DataFrame({'Rank':rank[1:],'Name of Billionaire':name[1:],'Net-worth': nw[1:],'Age':age[1:],'Citizenship':citizen[1:],'Source':source[1:],'Industry':ind[1:]})
billionaires_forbes

,Rank,Name of Billionaire,Net-worth,Age,Citizenship,Source,Industry
0,1.,Bernard Arnault & family,$233 B,75,France,LVMH,Fashion & Retail
1,2.,Elon Musk,$195 B,52,United States,"Tesla, SpaceX",Automotive
2,3.,Jeff Bezos,$194 B,60,United States,Amazon,Technology
3,4.,Mark Zuckerberg,$177 B,39,United States,Facebook,Technology
4,5.,Larry Ellison,$141 B,79,United States,Oracle,Technology
...,...,...,...,...,...,...,...
195,195.,Lei Jun,$10.9 B,54,China,Smartphones,Technology
196,195.,Georg Schaeffler,$10.9 B,59,Germany,Auto parts,Automotive
197,195.,Marcel Herrmann Telles & family,$10.9 B,74,Brazil,Beer,Food & Beverage
198,199.,David Velez & family,$10.8 B,42,Colombia,Fintech,Finance & Investments


In [97]:
driver.quit()

# 8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.


In [37]:
def get_youtube_comments(url):
    driver=webdriver.Chrome()
    driver.get(url)
    time.sleep(3)
    
    # scrape comments
    
    try: 
        scrolls = 0
        com=[]
        while len(com)<500 and scrolls<20:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            for comment in driver.find_elements(By.XPATH,'//div[@class="style-scope ytd-expander"]'):
                com.append(comment.text.strip())
                if len(com)>=500:
                    break
                
    # scrape upvote
        uv=[]
        while len(uv)<500 and scrolls<20:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            for u in driver.find_elements(By.XPATH,'//div[@id="toolbar"] /span'):
                uv.append(u.text)
                if len(uv)>=500:
                    break
                
    # scrape time
        tym=[]
        while len(tym)<500 and scrolls<20:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            for t in driver.find_elements(By.XPATH,'//span[@class="style-scope ytd-comment-view-model"]/a'):
                tym.append(t.text.strip())
                if len(tym)>=500:
                    break
                    
    except NoSuchElementException:
        print('Error: No element found')          
            
    if len(com)==len(uv)==len(tym):
        youtube_comments=pd.DataFrame({'Youtube comment':com,'Upvotes':uv,'Time of comment': tym})
        print (youtube_comments)
    
    driver.quit()


In [38]:
get_youtube_comments('https://www.youtube.com/watch?v=a6yCQdx3Pkg')

                                       Youtube comment Upvotes Time of comment
0    Saul again with perfect balance between entert...    1.5K    4 months ago
1    I would love a show with Chef Saul and Chef Fr...     681    4 months ago
2    I can't believe I get paid to do this. Love Sa...     363    4 months ago
3    Are there more videos with Saul? If not, there...     411    4 months ago
4    Chef Saul's spoon drop is the most wholesome t...      32    4 months ago
..                                                 ...     ...             ...
495  SAUL: Mexican Cooking Masterclass WHEN?? I wou...      16    4 months ago
496        Guys lets show Saul how big fanbase he have      64    4 months ago
497                     Saul is truly a gem of a human      15    4 months ago
498           Saul has the best personality and humor!      88    4 months ago
499  Saul is so lovely and adorable <3 you can tell...              7 days ago

[500 rows x 3 columns]


# 9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [63]:
# open chrome and load hostelworld
driver=webdriver.Chrome()
url='https://www.hostelworld.com/'
driver.get(url)

In [65]:
# search for location
enter=driver.find_element(By.XPATH,'//div[@class="destination-container force-focus"]/div/div/div[2]/input')
enter.send_keys('London')
time.sleep(1)

select=driver.find_element(By.XPATH,'/html/body/div[3]/div/div[2]/main/header/div/div[2]/div[1]/div[1]/div/div[1]/div[2]/div/ul/li[2]/button')
select.click()
time.sleep(1)

search=driver.find_element(By.XPATH,'//button[@class="large-button btn-content"]').click()

In [10]:
# scrape hostel name
name=[]
for n in driver.find_elements(By.XPATH,'//div[@ class="property-name"]/span'):
    name.append(n.text.strip())
print('Length : ',len(name) ,'\n\n', name)

Length :  30 

 ["Wombat's City Hostel London", 'Palmers Lodge - Swiss Cottage', 'NX London Hostel', 'Onefam Notting Hill', 'Urbany Hostel London', 'Generator London', 'Onefam Waterloo', "St Christopher's Village", 'Clink261', 'Safestay London Elephant & Castle', 'No.8 Seven Sisters', 'Smart Russell Square Hostel', 'Astor Hyde Park', 'Safestay London Kensington Holland Park', 'Queen Elizabeth Chelsea', 'Astor Museum Inn', 'Phoenix Hostel', 'Hostelle - women only hostel London', 'Smart Hyde Park Inn Hostel', 'Kabannas London St Pancras', 'Smart Camden Inn Hostel', 'Astor Victoria', 'Astor Kensington', 'Bell House Hostel', "St Christopher's Shepherds Bush", 'Park Villa', "St Christopher's Greenwich", 'PubLove @ The White Ferry, Victoria', 'Saint James Backpackers', 'Book a Bed Hostels']


In [11]:
# scrape hostel distance from city centre
dist=[]
for d in driver.find_elements(By.XPATH,'//div[@ class="property-distance"]/span[2]'):
    dist.append(d.text.strip('-'))
print('Length : ',len(dist) ,'\n\n', dist)

Length :  30 

 [' 3.6km from city centre', ' 6.5km from city centre', ' 6.1km from city centre', ' 5.5km from city centre', ' 5.4km from city centre', ' 3km from city centre', ' 0.7km from city centre', ' 1.8km from city centre', ' 3.2km from city centre', ' 1.7km from city centre', ' 9km from city centre', ' 2.6km from city centre', ' 4.3km from city centre', ' 5.8km from city centre', ' 5.7km from city centre', ' 2.2km from city centre', ' 4.2km from city centre', ' 5.1km from city centre', ' 5km from city centre', ' 3.3km from city centre', ' 4.4km from city centre', ' 1.8km from city centre', ' 4.9km from city centre', ' 4.2km from city centre', ' 7km from city centre', ' 6.3km from city centre', ' 7.6km from city centre', ' 2.4km from city centre', ' 5.5km from city centre', ' 6.9km from city centre']


In [19]:
# scrape hostel ratings
rate=[]
for r in driver.find_elements(By.XPATH,'//div[@class="rating-score"]/span[1]'):
    rate.append(r.text.strip())
    rates=rate[2:]
print('Length : ',len(rates) ,'\n\n', rates)

Length :  30 

 ['9.2', '8.7', '8.9', '9.7', '9.6', '7.5', '9.6', '7.6', '8.3', '7.6', '7.8', '7.6', '8.0', '7.2', '7.7', '8.4', '6.9', '8.6', '7.7', '8.4', '8.6', '7.4', '8.2', '7.2', '7.8', '9.3', '7.9', '7.8', '7.3', '6.3']


In [20]:
# scrape hostel total reviews
tr=[]
for t in driver.find_elements(By.XPATH,'//div[@class="review"]/span'):
    tr.append(t.text.strip())
print('Length : ',len(tr) ,'\n\n', tr)

Length :  30 

 ['(16255)', '(16101)', '(2289)', '(2741)', '(1204)', '(8178)', '(266)', '(12999)', '(594)', '(5320)', '(4231)', '(10258)', '(12374)', '(1843)', '(3757)', '(9490)', '(4689)', '(159)', '(6832)', '(3014)', '(3170)', '(15127)', '(6608)', '(165)', '(849)', '(933)', '(3452)', '(430)', '(1964)', '(1317)']


In [22]:
# scrape hostel overall reviews
ovr=[]
for o in driver.find_elements(By.XPATH,'//div[@class="rating-score"]/span[2]'):
    ovr.append(o.text.strip())
    over=ovr[2:]
print('Length : ',len(over) ,'\n\n', over)

Length :  30 

 ['Superb', 'Fabulous', 'Fabulous', 'Superb', 'Superb', 'Very Good', 'Superb', 'Very Good', 'Fabulous', 'Very Good', 'Very Good', 'Very Good', 'Fabulous', 'Very Good', 'Very Good', 'Fabulous', 'Good', 'Fabulous', 'Very Good', 'Fabulous', 'Fabulous', 'Very Good', 'Fabulous', 'Very Good', 'Very Good', 'Superb', 'Very Good', 'Very Good', 'Very Good', 'Good']


In [33]:
# scrape hostel privates from price
pv=[]
try:
    for p in driver.find_elements(By.XPATH,'//div[@class="property-accommodation-prices"]/div[1]'):
        if p:
            pv.append(p.text.strip().replace('\n',' '))
        else://div[@class="property-accommodation-prices"]/div[1]
            pv.append('-')
            
except NoSuchElementException:
    pv.append('-')
print('Length : ',len(pv) ,'\n\n', pv)

Length :  30 

 ['No Privates Available', 'No Privates Available', 'No Privates Available', 'Privates From €164', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available', 'No Privates Available']


In [42]:
# scrape hostel dorms from price
dv=[]
try:
    for dorm in driver.find_elements(By.XPATH,'//div[@class="property-accommodation-prices"]/div[2]'):
        if p:
            dv.append(dorm.text.strip().split()[-1])
        else:
            dv.append('-')
            
except NoSuchElementException:
    dv.append('-')
print('Length : ',len(dv) ,'\n\n', dv)

Length :  30 

 ['€57', '€34', '€30', '€55', '€42', '€48', '€60', '€28', '€50', '€36', '€14', '€27', '€36', '€18', '€30', '€49', '€22', '€46', '€24', '€57', '€46', '€28', '€29', '€29', '€20', '€21', '€43', '€22', '€35', '€53']


In [67]:
# get links for each searched hostel
lk=[]
for link in driver.find_elements(By.XPATH,'//div[@class="property-card"] /a'):
    lk.append(link.get_attribute('href'))
    links=lk[2:]
print('Length : ',len(links) ,'\n\n', links)

Length :  30 

 ['https://www.hostelworld.com/pwa/hosteldetails.php/Wombat-s-City-Hostel-London/London/88047?from=2024-04-20&to=2024-04-23&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/Palmers-Lodge-Swiss-Cottage/London/14348?from=2024-04-20&to=2024-04-23&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/NX-London-Hostel/London/15204?from=2024-04-20&to=2024-04-23&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/Onefam-Notting-Hill/London/291433?from=2024-04-20&to=2024-04-23&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/Urbany-Hostel-London/London/303427?from=2024-04-20&to=2024-04-23&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/Generator-London/London/510?from=2024-04-20&to=2024-04-23&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/Onefam-Waterloo/London/39493?from=2024-04-20&to=2024-04-23&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/St-Christopher-s-Village/London/502?from=2024-04-2

In [110]:
# scrape hostel facilities and property description
    
facilities=[]
descrip=[]
    
for i in links:
    driver.get(i)
    time.sleep(3)
    
    fac=[]
    desc=[]

   # scraping hostel facilities

    try:       
        
        for f in driver.find_elements(By.XPATH, '//ul[@class="facility-sections"]/li/ul/li'):
                text = f.text.strip()
                fac.append(text)
                
        facilities.append(','.join(fac) if fac else '-')


        # scraping property description
        
        try:
            description = driver.find_element(By.XPATH, '//div[@class="flex-80"]/div[6]/div/div[2]')
            desc.append(description.text.strip())
            
        except NoSuchElementException:
            desc.append('-')
            
        descrip.append(' '.join(desc) if desc else '-')

    except NoSuchElementException:
        facilities.append('-')
        descrip.append('-')


print(facilities, '\n\n')
print(descrip, '\n\n')

['Linen Included,Free City Maps,Free WiFi,Free Internet Access,Wheelchair Friendly,Security Lockers,Key Card Access,Common Room,Elevator,Bicycle Parking,BBQ,Breakfast Not Included,Meeting Rooms,Adaptors,Book Exchange,Hot Showers,Self-Catering Facilities,Dishwasher,Cooker,Fridge/Freezer,Utensils,Dryer,Outdoor Terrace,Reading Light,Hair Dryers For Hire,Iron / Ironing Board,Safe Deposit Box,Microwave,Washing Machine,Internet access,Laundry Facilities,Towels for hire,Luggage Storage,Fax Service,24 Hour Reception,Housekeeping,Internet café,Late check-out,Bar,Meals Available,Vending Machines,Cafe,Tea & Coffee Making Facilities,Board Games,Pool Table,Foosball,Wi-Fi', "Linen Included,Free WiFi,Free Internet Access,Security Lockers,Key Card Access,Common Room,Elevator,Bicycle Parking,BBQ,Breakfast Not Included,Adaptors,Book Exchange,Hot Showers,Self-Catering Facilities,Cooker,Fridge/Freezer,Utensils,Dryer,Outdoor Terrace,Reading Light,Hair Dryers For Hire,Parking,Hair Dryers,Iron / Ironing Boar

In [113]:
print(len(name),len(dist),len(rates),len(tr),len(over),len(pv),len(dv),len(links),len(facilities),len(descrip))

30 30 30 30 30 30 30 30 30 30


In [115]:
hostelworld_data=pd.DataFrame({'Hostel':name,'Distance from city':dist,'Rating':rates,'Total Reviews':tr,'Overall Reviews':over,'Privates from price':pv,'Dorms from price':dv,'Facilities':facilities,'Property Description':descrip})
hostelworld_data

,Hostel,Distance from city,Rating,Total Reviews,Overall Reviews,Privates from price,Dorms from price,Facilities,Property Description
0,Wombat's City Hostel London,3.6km from city centre,9.2,(16255),Superb,No Privates Available,€57,"Linen Included,Free City Maps,Free WiFi,Free I...",A safe haven in the middle of the metropolis: ...
1,Palmers Lodge - Swiss Cottage,6.5km from city centre,8.7,(16101),Fabulous,No Privates Available,€34,"Linen Included,Free WiFi,Free Internet Access,...","Whether you're a backpacker, flash-packer or a..."
2,NX London Hostel,6.1km from city centre,8.9,(2289),Fabulous,No Privates Available,€30,"Free Breakfast,Linen Included,Towels Included,...",-
3,Onefam Notting Hill,5.5km from city centre,9.7,(2741),Superb,Privates From €164,€55,"Linen Included,Free WiFi,Free Internet Access,...",-
4,Urbany Hostel London,5.4km from city centre,9.6,(1204),Superb,No Privates Available,€42,"Linen Included,Free WiFi,Security Lockers,Micr...",-
5,Generator London,3km from city centre,7.5,(8178),Very Good,No Privates Available,€48,"Linen Included,Free City Maps,Free WiFi,Free I...",Generator London is a design hotel-hostel loca...
6,Onefam Waterloo,0.7km from city centre,9.6,(266),Superb,No Privates Available,€60,"Linen Included,Free WiFi,Free Internet Access,...",-
7,St Christopher's Village,1.8km from city centre,7.6,(12999),Very Good,No Privates Available,€28,"Linen Included,Free City Maps,Free WiFi,Free I...",St Christopher's Inn at The Village in London ...
8,Clink261,3.2km from city centre,8.3,(594),Fabulous,No Privates Available,€50,"Linen Included,Free City Maps,Free WiFi,Key Ca...",Welcome to Clink261!\n\nClink261 is a comforta...
9,Safestay London Elephant & Castle,1.7km from city centre,7.6,(5320),Very Good,No Privates Available,€36,"Linen Included,Free WiFi,Security Lockers,Key ...",Safestay at Elephant & Castle is ideal if you ...


In [48]:
driver.quit()